In [1]:
import numpy as np

In [6]:
e1 = 0.3
e2 = 0.5
rho = 0.1
r = 10
d = 100*r

In [7]:
(1 - e1)**-rho

1.0363112099103142

In [8]:
(1 - 2*r/d)*(1 - e2)**-rho

1.0503379932855672

In [33]:
#Params
r = 10
d = r**3
n = d**4
n0 = d

eps1 = 0.3
eps2 = 0.1
eps3 = 0.01

a1 = a2 = a3 = d/3

In [34]:
def compute_k(pi1, pi2, pi3):
    return np.log(n0/n) * (pi1*(np.log(eps1**2 + (1-eps1)**2)) + \
                           pi2*(np.log(eps2**2 + (1-eps2)**2)) + \
                           pi3*(np.log(eps3**2 + (1-eps3)**2)))**-1

In [35]:
pi1s = np.linspace(0.01, 1, num=500)
pi2s = np.linspace(0.01, 1, num=500)
pi3s = np.linspace(0.01, 1, num=500)

In [36]:
k = compute_k(1/3, 1/3, 1/3)
(1 - r/d)**k

0.44099619128913287

In [37]:
best = 0
dist = None
for pi1 in pi1s:
    for pi2 in pi2s:
        pi3 = 1 - pi1 - pi2
        if pi3 < 0:
            continue
        
        k = compute_k(pi1, pi2, pi3)
        current = min((1 - 3*r*pi1/d)**k,\
                      (1 - 3*r*pi2/d)**k,\
                      (1 - 3*r*pi3/d)**k)
        if current > best:
            best = current
            dist = (pi1,pi2,pi3)
print(best)
print(dist)

0.44099134966344067
(0.3333867735470942, 0.3333867735470942, 0.3332264529058117)


# Improvement Calc

In [5]:
f = 1/20

In [6]:
(1 - (r/(d*f))) * ((0.5)*n)**-1

1.9600000000000003e-24

In [7]:
(1 - r/d)*( (1-f) * ((1-eps)*n)**-1 + (f) * ((0.5)*n)**-1)

1.068318367346939e-24

In [8]:
def compute_rho_u(nv):
    a = (1 - np.log(d**2)/np.log(nv))
    b = np.log(1 - (r)/(2*(1-f)*d))
    return a*b/ np.log(1-eps)
    #return (1)*np.log(1 - (r*f)/d) / np.log(1-eps)

In [9]:
def compute_uni_sum():
    total = 0
    for k in range(100):
        nv = n * (1-eps)**k * 0.5
        rho_u = compute_rho_u(nv)
        total += (1 - (r/(2*(1-f)*d)))**k * nv**(-rho_u) * (1 - f)**k * (f) * (1 - (r/(2*f*d)))
    return rho_u, total

In [10]:
#uni succ prob 
rho_f, psucc = compute_uni_sum()
psucc

0.4877502239075842

In [11]:
#uni error
(1-f)**100 * (1 - r/(d * (1-f)))**100 * (n * (1-eps)**100 * 0.5)**-rho_f

0.0027829529706102927

In [12]:
#opt prob 
rho_u = compute_rho_u(n/2)
(0.5*n)**(-rho_u) * (1 - (r/(2*f*d)))

0.49065516213209537

# Newest Approach

In [13]:
#Params
r = 1000
d = 1000*r
n = d**4

eps = 0.02

In [14]:
np.log(1 - r/d)/np.log(1-eps)

0.049523082122558716

In [15]:
(1 - r/d)*( (8/9) * ((1-eps)*n)**-1 + (1/9) * ((0.5)*n)**-1)

1.128122448979592e-24

In [16]:
(1 - (9*r)/d)*(n/2)**-1

1.9820000000000003e-24

In [17]:
E2 = (2/3)*np.log((1-eps)**2 + eps**2) + (1/3)*np.log((0.5)**2 + 0.5**2)
E2

-0.2577083989975574

In [18]:
rho_u = np.log(1 - r/d)/E2

In [19]:
#(1 - \frac{r}{d})^k (n(1-\eps)^k)^{-\rho_u}
for k in range(12):
    print(k)
    print((1 - r/d)**k * (n*(1-eps)**k)**(-rho_u))

0
0.8069096171296386
1
0.806165934959834
2
0.8054229381990987
3
0.8046806262157306
4
0.8039389983786098
5
0.803198054057198
6
0.8024577926215382
7
0.8017182134422539
8
0.8009793158905484
9
0.8002410993382052
10
0.7995035631575861
11
0.7987667067216315


# New Approach

In [22]:
#Params
r = 100
d = 100*r
n = d**6

In [23]:
#$\frac{1}{d} e^{-r/\ell}  (1 - \frac{r}{3d})^{k_o} (1 - \frac{r}{d})^{-k_o} \geq e^{-r}$
(1/d) * np.exp(-r/2) * (0.99)**20

1.5775378815392925e-26

In [24]:
np.exp(-r)

3.720075976020836e-44

In [25]:
eps1 = 0
eps2 = 0.01
eps3 = 0.5

In [26]:
# opt ln E[w_o]
E_o = np.log(eps3)
E_o

-0.6931471805599453

In [43]:
k_o = np.log(d/n)/E_o
(1 - (5*r*3)/(d))**k_o

2.0450076043096684e-05

In [44]:
#ln E[w_u]
E_u = np.log((1/3)*((1-eps1)**2 + eps1**2) + (1/3)*((1-eps2)**2 + eps2**2) + (1/3)*((1-eps3)**2 + eps3**2))
E_u

-0.1902730865815749

In [45]:
k_u = np.log(d**2 /n)/E_u
(1 - (5*r)/(d))**k_u

4.8614305700932185e-05

In [14]:
#E[ln w_u]
E2 = (1/3)*np.log((1-eps1)**2 + eps1**2) + (1/3)*np.log((1-eps2)**2 + eps2**2) + (1/3)*np.log((1-eps3)**2 + eps3**2)
E2

-0.2377152756888784

In [18]:
(1 - r/d)**(np.log(d/n)/E_u)

0.08781895974472351

In [740]:
pi1s = np.linspace(0.01, 1, num=100)
pi2s = np.linspace(0.01, 1, num=100)
pi3s = np.linspace(0.01, 1, num=100)

In [741]:
best = 1
dist = None
for pi1 in pi1s:
    for pi2 in pi2s:
        if pi1 + pi2 > 1:
            continue
        else:
            pi3 = 1 - pi1 - pi2
            
        E_o = pi1*np.log((1-eps1)**2 + eps1**2) + pi2*np.log((1-eps2)**2 + eps2**2) + \
                pi3*np.log((1-eps3)**2 + eps3**2)
        rho =  np.log(1 - r/d)/E_o
        if rho < best:
            best = rho
            dist = (pi1,pi2,pi3)

In [736]:
pi1,pi2,pi3 = dist
E_o = best
E_o

0.0014685839310322809

In [737]:
dist

(0.01, 0.01, 0.98)

In [101]:
np.log(n/d) / np.log(1/0.3) / 4

9.084324124617693

In [55]:
r = 50
d = 200
kmax = 1000
total = 0
for k in range(1, kmax+1):
    total += (1 - (r/d))**k * (0.5)**k
total

0.6000000000000003

In [102]:
np.log(0.5)/np.log(0.4)

0.7564707973660301

In [103]:
np.log(0.5)/np.log(0.3)

0.5757166424934449

In [54]:
(1 - (2*r)/d)

0.5

In [96]:
r = 1000
d = 100*r

d**(- (10/9) * np.log(1 - (r)/d) / np.log(1 - (r+1)/d)) / d**(-np.log(1 - (r)/d) / np.log(1 - (r+1)/d))

0.2786135560336649

In [74]:

rho = 0.2
e1 = 0.3
e2 = 0.5
e2**(1-rho) + (1-e2)**(1-rho) - (r/d)*(e2 ** (-rho))

1.1372113714470646

In [75]:
e1**(1-rho) + (1-e1)**(1-rho)

1.1334365376118631

# Old Approach

In [2]:
#Uniform

In [3]:
def comp_succ(rho, d, r, a1, e1, a2, e2, a3, e3, a4, e4):
    #psucc_opt_1 = (1 - r/d)*((1 - ((a1)*2*e1*(1-e1) + a2*2*e2*(1-e2) + a3*2*e3*(1-e3) + \
    #                                   a4*2*e4*(1-e4))/d))**(-rho)
    psucc_opt_1 = r/((a1)*2*e1*(1-e1) + a2*2*e2*(1-e2) + a3*2*e3*(1-e3) + a4*2*e4*(1-e4))
    return psucc_opt_1

In [4]:
def compute_rho(d, r, a1, e1, a2, e2, a3, e3, a4, e4):
    upper = 1.5
    lower = 0
    thresh = 1e-6
    
    mid = (upper + lower)/2
    
    i=0
    while (abs(comp_succ(mid, d, r, a1, e1, a2, e2, a3, e3, a4, e4) - 1) > thresh):
        #if (i%1000 == 0):
        #    print(mid)
        if comp_succ(mid, d, r, a1, e1, a2, e2, a3, e3, a4, e4) > 1:
            upper = mid
        else:
            lower = mid
        mid = (upper + lower)/2
        
        if abs(mid - 1.5) < 0.05:
            return 1.5
        
        i+=1
    return mid


In [5]:
#Opt1 - search for best p1,p2,p3

In [6]:
def search_ps(rho, d, r, a1, e1, a2, e2, a3, e3, a4, e4):
    max_psucc = 0
    max_dists = None
    for p1 in np.linspace(0, 30/d, num=100):
        for p2 in np.linspace(0, 30/d, num=100):
            for p3 in np.linspace(0, 30/d, num=100):
                current = a1*p1 + a2*p2 + a3*p3

                if (current < 1):
                    p4 = (1 - current)/a4
                    psucc_opt_1 = (a1*p1*(1-e1)**(-rho) + a2*p2*(1-e2)**(-rho) + 
                                   (a3-r)*p3*(1-e3)**(-rho) + (a4)*p4*(1-e4)**(-rho))

                    psucc_opt_2 = (a1*p1*(1-e1)**(-rho) + (a2-r)*p2*(1-e2)**(-rho) + 
                                   (a3)*p3*(1-e3)**(-rho) + (a4)*p4*(1-e4)**(-rho))

                    psucc_opt_3 = ((a1-r)*p1*(1-e1)**(-rho) + (a2)*p2*(1-e2)**(-rho) + 
                                   (a3)*p3*(1-e3)**(-rho) + (a4)*p4*(1-e4)**(-rho))

                    psucc_opt_4 = ((a1)*p1*(1-e1)**(-rho) + (a2)*p2*(1-e2)**(-rho) + 
                                   (a3)*p3*(1-e3)**(-rho) + (a4-r)*p4*(1-e4)**(-rho))
                else:
                    continue

                psucc_opt = min(psucc_opt_1,psucc_opt_2,psucc_opt_3,psucc_opt_4)

                if psucc_opt > max_psucc:
                    max_psucc = psucc_opt
                    max_dists = (p1,p2,p3,p4)
    
    p1 = p2 = p3 = 0
    p4 = 1/a4
    
    psucc_opt_1 = (a1*p1*(1-e1)**(-rho) + a2*p2*(1-e2)**(-rho) + 
                                   (a3-r)*p3*(1-e3)**(-rho) + (a4)*p4*(1-e4)**(-rho))
    psucc_opt_2 = (a1*p1*(1-e1)**(-rho) + (a2-r)*p2*(1-e2)**(-rho) + 
                   (a3)*p3*(1-e3)**(-rho) + (a4)*p4*(1-e4)**(-rho))

    psucc_opt_3 = ((a1-r)*p1*(1-e1)**(-rho) + (a2)*p2*(1-e2)**(-rho) + 
                   (a3)*p3*(1-e3)**(-rho) + (a4)*p4*(1-e4)**(-rho))

    psucc_opt_4 = ((a1)*p1*(1-e1)**(-rho) + (a2)*p2*(1-e2)**(-rho) + 
                   (a3)*p3*(1-e3)**(-rho) + (a4-r)*p4*(1-e4)**(-rho))
    
    print("uniform on balanced bits")
    #print(p4)
    #print(psucc_opt_1, psucc_opt_2,psucc_opt_3,psucc_opt_4)
    return (max_psucc, max_dists)

### Advantage Computation

In [7]:


rad=20
alpha1=1500
alpha2=1500
alpha3=0
alpha4=1500
dim = alpha1 + alpha2 + alpha3 + alpha4
alpha1s = [alpha1]
alpha2s = [alpha2]
alpha3s = [alpha3]
alpha4s = [alpha4]

eps1=0
eps2=0.05
eps3=0.05
eps4=0.5
eps1s = [eps1]
eps2s = [eps2]
eps3s = [eps3]
eps4s = [eps4]


In [8]:
print(rad/((alpha1)*2*eps1*(1-eps1) + alpha2*2*eps2*(1-eps2) + alpha3*2*eps3*(1-eps3) + alpha4*2*eps4*(1-eps4)))

0.022408963585434174


In [9]:
delta = 0.001

In [10]:
# rho is solution to :
# (1 - rad/dim)(1 - alpha1*2*eps1*(1-eps1)+alpha2*2*eps2*(1-eps2)+alpha3*2*eps3*(1-eps3) / dim)**-rho = 1
rho_u = compute_rho(dim, rad, alpha1, eps1, alpha2, eps2, alpha3, eps3, alpha4, eps4)
rho_u
alpha1*2*eps1*(1-eps1)+alpha2*2*eps2*(1-eps2)+alpha3*2*eps3*(1-eps3)+alpha4*2*eps4*(1-eps4)

892.5

In [20]:
def compute_advantages(delta, r,a1s,a2s,a3s,a4s,e1s,e2s,e3s,e4s):
    advantages = []
    max_adv = 0
    for a1 in a1s:
        for a2 in a2s:
            for a3 in a3s:
                for a4 in a4s:
                    for e1 in e1s:
                        for e2 in e2s:
                            for e3 in e3s:
                                for e4 in e4s:
                                    d = a1 + a2 + a3 + a4
            
                                    #rho=r/((a1)*2*e1*(1-e1) + a2*2*e2*(1-e2) + a3*2*e3*(1-e3) + a4*2*e4*(1-e4))
                                    rho = 0.019
                                    psucc_opt = search_ps(rho, d, r, a1, e1-delta, a2, 
                                                          e2-delta, a3, e3-delta, a4, e4-delta)
                                    if psucc_opt[0] > max_adv:
                                        max_adv = psucc_opt[0]
                                        advantages.append([psucc_opt, a1, e1, a2, e2, a3, e3, a4, e4, rho])
    return advantages

In [21]:
advs = compute_advantages(delta,rad,alpha1s,alpha2s,alpha3s,alpha4s, eps1s,eps2s,eps3s, eps4s)
advs[-1]

uniform on balanced bits


[(1.0002843629064149,
  (0.0, 0.00033670033670033677, 0.0, 0.00032996632996632987)),
 1500,
 0,
 1500,
 0.05,
 0,
 0.05,
 1500,
 0.5,
 0.019]

In [1420]:
1.002067049835572**8

1.0166565300101607

## Whole Dataset

In [618]:
d = advs[-1][1] + advs[-1][3] + advs[-1][5]
rho = advs[-1][-1]

In [1430]:
1/1500

0.0006666666666666666

In [754]:
advs[-1]

[(1.1449360115926936, (0.0, 0.0, 0.005)), 300, 0, 0, 0.1, 200, 0.4, 1.5]

In [755]:
#k is solution to n(1-e3-delta)**k = (k ln d)/delta^2

In [757]:
#How does advantage scale? 
#n k adv
#2*10**10, 1.006214 * (2 \cdot 10^{10})^{-\rho_u}
#10**11, 1.01038 * 10^{-11\rho_u}
#5*10**11, 1.01456 * (5 \cdot 10^{11})^{-\rho_u}
#2*10**12, 1.0167 * (2\cdot 10^{12})^{-\rho_u}

In [ ]:
#Transform data to log(y) = -rho * log(n)
# The 4 points become
# log(n) log(y)
# log(2*10**10) log()
# log(10**11)
# log(5*10**11)
# log(2*10**12)

In [1424]:
rho_u = 0.022408963585434174

In [1428]:
advs = np.array([1.006214, 1.01038, 1.01456, 1.0167])
ns = np.array([2*10**10,10**11, 5*10**11, 2*10**12])

xs = np.log(ns).reshape(-1, 1)
ys = np.log(advs * np.power(ns,-rho_u))

In [1429]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score


regr = linear_model.LinearRegression()
regr.fit(xs, ys)
rho = -regr.coef_
rho

array([0.02011646])

In [796]:
(advs[-1][0][0])**20.39

15.796067554499029